In [1]:
import pandas as pd
import numpy as np

In [2]:
pd.set_option('display.max_columns', None)

In [3]:
df_data_base = pd.read_csv("../data/originals/PROD_TERMIN_ART_SIN_9.csv")
df_data_base_imp = pd.read_csv("../data/originals/FACTOR_IMPACTO.csv")
df_data_new = pd.read_csv("../data/process/all_data_api.csv")

In [4]:
df_data_base["TITULO"] = df_data_base["TITULO"].str.lower().str.strip().str.replace(":", "").str.replace("-", "")
df_data_new["title"] = df_data_new["title"].str.lower().str.strip().str.replace(":", "").str.replace("-", "")

In [5]:
df_data_new["TITULO"] = df_data_new["title"]

In [6]:
result = pd.merge(df_data_base, df_data_new, on="TITULO", how="inner")

In [7]:
result.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2355 entries, 0 to 2354
Data columns (total 83 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   IDDECINVPROYECTOPRODTERMIN    2355 non-null   int64  
 1   IDDECINVPROYECTO              2355 non-null   float64
 2   IDDECINVTIPOPRODUCTOESPERADO  2355 non-null   int64  
 3   TITULO                        2355 non-null   object 
 4   ANIO                          2355 non-null   int64  
 5   VOLUMEN                       2324 non-null   float64
 6   PAGINAS                       2316 non-null   object 
 7   NOMBREREVISTA                 2244 non-null   object 
 8   ISINDEX                       2257 non-null   float64
 9   BDREVISTA                     2149 non-null   float64
 10  ISSN                          2136 non-null   object 
 11  FACTIMPACTO                   591 non-null    object 
 12  EDITORIAL                     84 non-null     object 
 13  EST

In [8]:
result["TITULO"].value_counts()

TITULO
building academic branding the digital branding as academic footprint                                                                                                           4
adaptive evacuation management system based on monitoring techniques                                                                                                            4
humanmachine musical composition in realtime based on emotions through a fuzzy logic approach                                                                                   4
overcoming limitations of polymer additive manufacturing for snapfit joint design and part assembly                                                                             4
development of sociodemographic indicators for modeling the household solid waste generation in guayaquil (ecuador) quantification, characterization and energy valorization    3
                                                                                                       

In [9]:
result[result["title"] == "Preface"]

,IDDECINVPROYECTOPRODTERMIN,IDDECINVPROYECTO,IDDECINVTIPOPRODUCTOESPERADO,TITULO,ANIO,VOLUMEN,PAGINAS,NOMBREREVISTA,ISINDEX,BDREVISTA,ISSN,FACTIMPACTO,EDITORIAL,ESTADOPUBLICACION,ESPECIFICACION,URL,ULTIMO_CAMBIO,NEVENTO,LEVENTO,FEVENTO,ADJUNTO,FECHAPUBLICACION,RESUMEN,KEYS,ASIST_OBJETIV,INNOVA_VERIFIC,LOGROS_SISTEMAC,RETOS_COMUNIC,IDUNIDAD,TIPOLAB,MARCA,MODELO,SERIE,INVENTARIO,ESTADO,FRECUENCIA,FVENCIMIENTO,SJR,CUARTIL,IDDECINVREVISTA,APROBADO,IDDECINVMOTIVOPRODTERMIN,OBSERVACIONARPOB,IDPUBLICACIONPERSONA,ADJUNTO2,ADJUNTO3,ADJUNTO4,CODIGOPRODUCTO,TIPOTESIS,REVISIONPARES,DOI,INVENTOR,SUBCLASE,URLREVISTA,MATERIAINTEGRADORA,NUMEROSOLICITUD,OBJGENERAL,ESTAELIMINADO,url,doi,url_doi,eid,title,abstract,key_words,cuartil,volume,authors,affiliations,issn,magazine,citeScore,citations_scopus_count,language,coverDate,publication_year,page_raw,page_count,scimago_factor_SJR,scimago_SJR_Best_Quartile,eid_original_input,error_message,año


In [10]:
sql_update_commands = []
TABLE_NAME = "ESPOL.TBL_DEC_INV_PROYECTO_PROD_TERMIN"

# Mapeo de las columnas de origen (df_data_new) a las de destino (df_data_base)
column_mapping = {
    'doi': 'DOI',
    'issn': 'ISSN',
    'page_raw': 'PAGINAS',
    'abstract': 'RESUMEN',
    'key_words': 'KEYS',
    'volume': 'VOLUMEN'
}

In [11]:
# Función para escapar caracteres especiales en SQL (especialmente comillas simples)
def escape_sql(value):
    if value is None or pd.isna(value):
        return "NULL"
    # Reemplaza una comilla simple ' por dos comillas simples ''
    return str(value).replace("'", "''")

In [12]:
# Función para validar si un valor puede convertirse a entero
def is_valid_integer(value):
    try:
        int(value)
        return True
    except (ValueError, TypeError):
        return False

In [13]:
print("Procesando registros para generar script de actualización...")

# Iteramos sobre cada fila del DataFrame resultante del merge
for index, row in result.iterrows():
    # Obtenemos el ID para la cláusula WHERE. Si no hay ID, no podemos actualizar.
    record_id = row['IDDECINVPROYECTOPRODTERMIN']
    if pd.isna(record_id):
        print(f"  - Omitiendo fila con título '{row['title']}' porque no se encontró un IDDECINVPROYECTOPRODTERMIN correspondiente.")
        continue

    # Lista para guardar las partes SET de la consulta para este registro específico
    set_clauses = []

    # Aplicamos la lógica de actualización para cada campo
    for source_col, dest_col in column_mapping.items():
        new_value = row[source_col]
        old_value = row[dest_col]

        # La condición clave:
        # Si el valor antiguo (de la BD) es NULO/VACÍO y el valor nuevo SÍ tiene contenido...
        if pd.isna(old_value) and pd.notna(new_value):
            # Validación especial para VOLUMEN
            if dest_col == 'VOLUMEN':
                if is_valid_integer(new_value):
                    # Si es válido como entero, lo convertimos
                    set_clauses.append(f"{dest_col} = {int(new_value)}")
                else:
                    # Si no es válido, lo ignoramos y mostramos el mensaje
                    print(f"  ! Ignorando VOLUMEN para DOI '{row['doi']}': valor '{new_value}' no es entero válido")
                    continue
            else:
                # Para otros campos, aplicamos escape normal
                set_clauses.append(f"{dest_col} = '{escape_sql(new_value)}'")

    # Si hay al menos un campo para actualizar en este registro...
    if set_clauses:
        # ...construimos la sentencia UPDATE completa.
        update_statement = f"UPDATE {TABLE_NAME} SET {', '.join(set_clauses)} WHERE IDDECINVPROYECTOPRODTERMIN = {int(record_id)};"
        sql_update_commands.append(update_statement)
        print(f"  + Generando UPDATE para ID: {int(record_id)}")

Procesando registros para generar script de actualización...
  + Generando UPDATE para ID: 2304
  + Generando UPDATE para ID: 2306
  + Generando UPDATE para ID: 2336
  + Generando UPDATE para ID: 2338
  + Generando UPDATE para ID: 2354
  + Generando UPDATE para ID: 2355
  ! Ignorando VOLUMEN para DOI '10.1109/CVPRW50498.2020.00041': valor '2020-June' no es entero válido
  + Generando UPDATE para ID: 3296
  + Generando UPDATE para ID: 2388
  + Generando UPDATE para ID: 2402
  + Generando UPDATE para ID: 2405
  + Generando UPDATE para ID: 2553
  + Generando UPDATE para ID: 2449
  + Generando UPDATE para ID: 2512
  + Generando UPDATE para ID: 2515
  + Generando UPDATE para ID: 2518
  + Generando UPDATE para ID: 2520
  ! Ignorando VOLUMEN para DOI '10.18687/LACCEI2018.1.1.78': valor '2018-July' no es entero válido
  + Generando UPDATE para ID: 2539
  + Generando UPDATE para ID: 2545
  + Generando UPDATE para ID: 2570
  + Generando UPDATE para ID: 2571
  + Generando UPDATE para ID: 2572
  +

  + Generando UPDATE para ID: 3076
  + Generando UPDATE para ID: 3098
  + Generando UPDATE para ID: 3103
  + Generando UPDATE para ID: 3117
  + Generando UPDATE para ID: 3118
  + Generando UPDATE para ID: 3119
  + Generando UPDATE para ID: 3120
  + Generando UPDATE para ID: 3127
  + Generando UPDATE para ID: 3128
  + Generando UPDATE para ID: 3129
  + Generando UPDATE para ID: 3188
  + Generando UPDATE para ID: 3190
  + Generando UPDATE para ID: 3210
  + Generando UPDATE para ID: 3256
  ! Ignorando VOLUMEN para DOI '10.1109/CVPRW50498.2020.00056': valor '2020-June' no es entero válido
  + Generando UPDATE para ID: 3291
  ! Ignorando VOLUMEN para DOI '10.1109/IWSSIP48289.2020.9145121': valor '2020-July' no es entero válido
  + Generando UPDATE para ID: 3293
  + Generando UPDATE para ID: 3308
  + Generando UPDATE para ID: 3312
  + Generando UPDATE para ID: 3328
  + Generando UPDATE para ID: 3336
  + Generando UPDATE para ID: 3414
  + Generando UPDATE para ID: 3433
  + Generando UPDATE pa

Escritura del archivo .sql final

In [14]:
output_filename = 'update_script.sql'
with open(output_filename, 'w', encoding='utf-8') as f:
    if not sql_update_commands:
        f.write("-- No se encontraron registros que requieran actualización según las reglas especificadas.\n")
    else:
        f.write(f"-- Script de actualización generado automáticamente.\n")
        f.write(f"-- Total de registros a actualizar: {len(sql_update_commands)}\n\n")
        for command in sql_update_commands:
            f.write(command + '\n')

print(f"\n¡Proceso completado! El script de actualización se ha guardado en '{output_filename}'")


¡Proceso completado! El script de actualización se ha guardado en 'update_script.sql'
